In [18]:
!pip install lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 27.0 MB/s  0:00:00 eta 0:00:01


In [1]:
import pandas as pd
import os

In [16]:
def read_xml_files(folder_path):
    """Read all XML files in a folder and return a concatenated DataFrame."""
    # List all XML files in the folder
    xml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')]
    
    # Read each XML file and append to a list
    df_list = []
    for xml_file in xml_files:
        file_path = os.path.join(folder_path, xml_file)
        df = pd.read_xml(file_path)
        df_list.append(df)

    print(len(df_list), "XML files read.")
    
    # Concatenate all DataFrames in the list
    concatenated_df = pd.concat(df_list, ignore_index=True)
    
    return concatenated_df

In [14]:
print(os.getcwd())

/Users/s2087805/Downloads/nls-trial-catalogue-published-material


In [19]:
data = read_xml_files('../nls-catalogue-published-material/nls-catalogue-published-material_dc')


51 XML files read.


In [20]:
data

,title,creator,type,publisher,date,language,subject,description,rights,identifier,relation,coverage,format
0,Anglo-russkii slovar' po mikroelektronike : ok...,"Prokhorov, K. IA.(Konstantin IAkovlevich)",text,Moskva Russkii iazyk,1985,rus,Microelectronics,None,None,None,None,None,None
1,Local government finance in Scotland: rate sup...,Great Britain.Scottish Office.,text,[Edinburgh] Scottish Office,1985,eng,Local finance,None,None,None,None,None,None
2,The correlation of monthly rainfall with posit...,Commonwealth Scientific and Industrial Researc...,text,Melbourne : Commonwealth Scientific and Indust...,1958.,eng,Rainfall anomalies,In a folder with thirteen folded maps.,None,None,None,None,None
3,Catalogue,Peter Deloford (Bookseller),text,London P. Deloford,[1974]-,eng,Antiquarian booksellers,None,None,None,None,None,None
4,Control of pests and diseases of grass and for...,None,text,"Alnwick MAFF, ADAS",1985,eng,Grasses,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5091422,Textbook of fish culture breeding and cultivat...,"Kahn, Henry.",text,London Fishing News,[1973],eng,Fish-culture.,Bibliographyp.425-436,None,None,None,None,None
5091423,Tax planning a service or crime?,"Carmichael, K. S.(Keith Stanley).",text,London Institute of Chartered Accountants in E...,[1969],eng,Tax accounting,None,None,None,None,None,None
5091424,Tables for use in high resolution mass spectro...,"Cleaver, R. L.",text,London Heyden,1970,eng,Mass spectrometry,Notebook format. [28]p. booklet in pocket. Pag...,None,None,None,None,None
5091425,A systems approach to community college education,"Barbee, David E.",text,Princeton [Croydon] Auerbach,1972,eng,Municipal junior colleges,Bibliographyp.161-174. - Includes index,None,None,None,None,None


In [21]:
data.language.describe()

count     5091057
unique        367
top           eng
freq      4171764
Name: language, dtype: object

In [22]:
data.language.unique() 

array(['rus', 'eng', 'und', 'lat', 'iri', '   ', 'pol', 'wel', 'fre',
       'ger', 'zxx', 'ita', 'spa', 'hrv', 'per', 'swe', 'ewo', 'ara',
       'gla', 'dut', 'slo', 'sco', 'gre', 'nor', 'mul', 'alb', 'dan',
       'scc', 'bul', 'mis', 'cor', 'por', 'chi', 'lit', 'jpn', 'grc',
       'heb', 'rum', 'bre', 'gle', 'epo', 'hun', 'cat', 'slv', 'enm',
       'tur', 'ice', 'ang', 'guj', 'ben', 'kor', 'd  ', 'snh', 'baq',
       'kon', 'tog', 'nic', 'ful', 'fao', 'fri', 'cze', 'frm', 'may',
       'swa', 'ukr', 'asm', '|||', 'nya', 'kan', 'syr', 'fin', 'arm',
       'esp', 'twi', 'vie', 'gaa', 'ind', 'oji', 'kur', None, 'srp',
       'yor', 'afr', 'tha', 'mon', 'fro', 'mao', 'men', 'zul', 'bem',
       'mac', 'inc', 'est', 'yao', 'urd', 'mal', 'kin', 'cam', 'gal',
       'btk', 'esk', 'ton', 'ada', 'pan', 'art', 'max', 'int', 'pli',
       'aze', 'hin', 'glg', 'lan', 'som', 'map', 'roa', 'eth', 'mlt',
       'pro', 'lug', 'tam', 'bel', 'smi', 'tib', 'fat', 'ijo', 'sho',
       '|d', 'lav', '

In [ ]:
import numpy as np
import re
from collections import Counter
def clean_and_analyze(df):
    stats = {}
    for col in df.columns:
        col_data = df[col]
        # Replace common missing value indicators with np.nan
        df[col] = col_data.replace(['', 'NA', 'N/A', 'None', None, 'null', 'Null', 'NULL', 'nan', 'NaN'], np.nan)
        missing_count = df[col].isna().sum()
        unique_count = df[col].nunique(dropna=True)
        value_counts = df[col].value_counts(dropna=True)
        messy = []
        outliers = []
        # Clean date columns to only keep years
        if 'date' in col.lower() or 'year' in col.lower():
            df[col] = df[col].astype(str).str.extract(r'(\d{4})')
            messy = col_data[~col_data.astype(str).str.match(r'^\[?\d{4}\]?$')].unique().tolist()
        # Clean string columns: trim whitespace, lower case, remove special chars
        elif df[col].dtype == object:
            df[col] = df[col].astype(str).str.strip().str.lower().str.replace(r'[^\w\s]', '', regex=True)
            messy = col_data[df[col].astype(str).str.contains(r'[^\w\s]', regex=True)].unique().tolist()
        # Detect outliers for numeric columns
        if np.issubdtype(df[col].dropna().dtype, np.number):
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            outliers = df[col][(df[col] < q1 - 1.5*iqr) | (df[col] > q3 + 1.5*iqr)].unique().tolist()
        # Suggest additional cleaning/stats
        # 1. Check for duplicate rows
        # 2. Check for columns with single unique value (uninformative)
        # 3. Check for columns with high cardinality (many unique values)
        # 4. Check for columns with mixed types (e.g., numbers and strings)
        mixed_types = len(set(type(x) for x in df[col].dropna())) > 1
        high_cardinality = unique_count > 1000
        single_value = unique_count == 1
        stats[col] = {
            'unique_count': unique_count,
            'value_counts': value_counts.to_dict(),
            'missing_count': missing_count,
            'messy_data': messy,
            'outliers': outliers,
            'mixed_types': mixed_types,
            'high_cardinality': high_cardinality,
            'single_value': single_value
        }
        print(f"Column: {col}")
        print(f"  Unique values: {unique_count}")
        print(f"  Missing values: {missing_count}")
        print(f"  Messy data: {messy}")
        print(f"  Outliers: {outliers}")
        print(f"  Mixed types: {mixed_types}")
        print(f"  High cardinality: {high_cardinality}")
        print(f"  Single value: {single_value}")
        print(f"  Value frequencies: {value_counts.to_dict()}")
        print("-" * 40)
    # Additional suggestions:
    # - Remove duplicate rows: df.drop_duplicates(inplace=True)
    # - Normalize categorical values (e.g., country names, language codes)
    # - Standardize formats (e.g., phone numbers, emails)
    # - Parse and validate structured fields (e.g., ISBN, identifiers)
    # - Impute missing values if needed
    return df, stats
data_cleaned, data_stats = clean_and_analyze(data)